In this file we will compute molecular fingerprints as bit vectors and save them in a dataframe

First we import the necessary modules

In [60]:
#import basic python packages
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import sklearn packages
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, f1_score, matthews_corrcoef
from scipy.stats import randint

#import rdkit packages
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True
from rdkit.Chem import Descriptors, AllChem, PandasTools

First we read the file moleculus combined, which contains the smiles string and ALDH1_inhibition

In [61]:
filename = 'Molecules_combined.csv'
df = pd.read_csv(filename)
df.head()

,SMILES,ALDH1_inhibition
0,COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...,1
1,O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1,1
2,Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...,1
3,CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1,1
4,CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21,1


Now we add a new column with RDKit molecules and switch the columns around

In [62]:
PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule')
df = df[['SMILES','Molecule','ALDH1_inhibition']]
df.head()

,SMILES,Molecule,ALDH1_inhibition
0,COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...,<rdkit.Chem.rdchem.Mol object at 0x000001EB919...,1
1,O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1,<rdkit.Chem.rdchem.Mol object at 0x000001EB919...,1
2,Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...,<rdkit.Chem.rdchem.Mol object at 0x000001EB97C...,1
3,CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1,<rdkit.Chem.rdchem.Mol object at 0x000001EB97C...,1
4,CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21,<rdkit.Chem.rdchem.Mol object at 0x000001EB97C...,1


Check whether all smiles strings could be converted to molecules

In [63]:
#Print the amount of 0 values in the molecule column.
df.Molecule.isna().sum()

0

No 0 values so no need to drop any rows
Now we use the RDKit molecule objects to calculate the morgan fingerprint of each molecule

First we make a list of molecules. Then we turn this into a list of morgan fingerprint (radius =2) bitvectors and store them in a new dataframe

In [64]:
#create list of molecules from dataframe
mol_list = df['Molecule'].tolist()

#Calculate morgan fingerprints 
morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in mol_list]

#Store the bits of the morgan fingerprint in a new dataframe
morgan_fp_list = [list(i) for i in morgan_fp]
fp_df = pd.DataFrame(morgan_fp_list)

#Rename columns
bitnumbers = [i for i in range(1,2049)]
fp_df.columns = bitnumbers
fp_df.head()

,1,2,3,4,5,6,7,8,9,10,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we create a new dataframe combining the smiles strings and the morgan fingerprint bits and save this as a csv file

In [65]:
smiles_column_df = df.iloc[:,0].to_frame()
df_smiles_fp = smiles_column_df.merge(fp_df,left_index=True, right_index=True)
df_smiles_fp.head()
df_smiles_fp.to_csv('desciptors_fingerprints.csv')

Add the 10 descriptors with the highest loading for pc1 we found in Data Preperation for ML

In [98]:
descriptors = []

for mol in mol_list:
    descriptors.append([Descriptors.MaxEStateIndex(mol),
                        Descriptors.fr_C_O_noCOO(mol),
                        Descriptors.BCUT2D_LOGPLOW(mol),
                        Descriptors.BCUT2D_MRLOW(mol),
                        Descriptors.SMR_VSA1(mol),
                        Descriptors.EState_VSA10(mol),
                        Descriptors.VSA_EState2(mol),
                        Descriptors.SlogP_VSA2(mol),
                        Descriptors.MolMR(mol),
                        Descriptors.SlogP_VSA5(mol)])
descriptors = np.asarray(descriptors)
descriptors.shape

(2000, 10)

add these descriptors to a new dataframe

In [100]:
df_descriptors = pd.DataFrame(descriptors)
df_descriptors.head()

,0,1,2,3,4,5,6,7,8,9
0,13.083531,2.0,-2.524368,-0.128181,14.325937,9.589074,26.972964,64.304606,126.8344,25.328832
1,12.170097,1.0,-2.240774,-0.118316,9.211688,4.794537,21.295170,26.195090,104.3507,11.323699
2,10.905837,0.0,-3.124535,-0.951912,19.160451,5.106527,5.789600,49.553366,129.8585,55.442513
3,11.562446,0.0,-2.211289,0.556316,4.417151,4.794537,13.738424,13.089513,78.7550,13.847474
4,12.108866,1.0,-2.269383,-0.115075,13.212334,13.212334,17.622350,32.109481,110.0965,10.440599


In [106]:
df_fp_descr = pd.concat([df_smiles_fp, df_descriptors], axis=1)
df_fp_descr = df_fp_descr.drop(columns=['SMILES'])
df_fp_descr.head()

,1,2,3,4,5,6,7,8,9,10,...,0,1,2,3,4,5,6,7,8,9
0,0,1,0,0,0,0,0,0,0,0,...,13.083531,2.0,-2.524368,-0.128181,14.325937,9.589074,26.972964,64.304606,126.8344,25.328832
1,0,0,0,0,0,0,0,0,0,0,...,12.170097,1.0,-2.240774,-0.118316,9.211688,4.794537,21.295170,26.195090,104.3507,11.323699
2,0,1,0,0,0,0,0,0,0,0,...,10.905837,0.0,-3.124535,-0.951912,19.160451,5.106527,5.789600,49.553366,129.8585,55.442513
3,0,0,0,0,0,0,0,0,0,0,...,11.562446,0.0,-2.211289,0.556316,4.417151,4.794537,13.738424,13.089513,78.7550,13.847474
4,0,0,0,0,0,0,0,0,0,0,...,12.108866,1.0,-2.269383,-0.115075,13.212334,13.212334,17.622350,32.109481,110.0965,10.440599


In [107]:
scaler = MinMaxScaler()
df_fp_descr[:] = scaler.fit_transform(df_fp_descr[:])
df_fp_descr.head()

,1,2,3,4,5,6,7,8,9,10,...,0,1,2,3,4,5,6,7,8,9
0,0,1,0,0,0,0,0,0,0,0,...,0.861052,0.4,0.451215,0.310203,0.266167,0.210653,0.379999,0.647103,0.795574,0.278653
1,0,0,0,0,0,0,0,0,0,0,...,0.788705,0.2,0.630917,0.313816,0.171148,0.105327,0.303811,0.263603,0.617928,0.124577
2,0,1,0,0,0,0,0,0,0,0,...,0.688571,0.0,0.070915,0.008557,0.355989,0.112180,0.095746,0.498660,0.819468,0.609947
3,0,0,0,0,0,0,0,0,0,0,...,0.740577,0.0,0.649601,0.560862,0.082068,0.105327,0.202409,0.131721,0.415693,0.152342
4,0,0,0,0,0,0,0,0,0,0,...,0.783855,0.2,0.612789,0.315002,0.245477,0.290249,0.254526,0.323120,0.663326,0.114861


Now we will assign the discriptors to the model parameter X and the ALDH1_Inhibition to model parameter y. We will then proceed to create a 80%-20% train test split with a set seed for reproducebility

In [109]:
X = df_fp_descr
Y = df['ALDH1_inhibition']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Check the distribution between inhibiter and non inhibiters.

In [110]:
sum(Y) / len(Y)

0.3

In [115]:
rf_model = RandomForestClassifier(max_depth=18,n_estimators=129)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=18, n_estimators=129)

In [116]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy=", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1_score=", f1)


Accuracy= 0.82
Precision: 0.7448979591836735
Recall: 0.6083333333333333
f1_score= 0.6697247706422018


In [117]:
y_pred_train = rf_model.predict(X_train)
training_accuracy = accuracy_score(y_train, y_pred_train)
print('Accuracy on training set:',training_accuracy)

Accuracy on training set: 0.99625


Tuning Hyperparamers

In [114]:
parm_distribution = {'n_estimators': randint(50,500),
                    'max_depth': randint(1,20)}
rf_model = RandomForestClassifier()
randomized_parm_optimization = RandomizedSearchCV(rf_model,
                                                  param_distributions=parm_distribution,
                                                  scoring= 'f1',
                                                  n_iter=6,
                                                  cv=5)
randomized_parm_optimization.fit(X_train,y_train)

top_rf_model = randomized_parm_optimization.best_estimator_
print(top_rf_model)

RandomForestClassifier(max_depth=18, n_estimators=129)
